## 1. Import:

### 1.1 Modules:

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
# new stuff for playing around:
import sweetviz as sv # Epic feature analysis as .html
# from dataprep.eda import plot_correlation # Neat correlation plotting
# from lazypredict.Supervised import LazyRegressor, LazyClassifier # Uber-modelling
# sklearn:
import sklearn
from sklearn import set_config
from sklearn.datasets import make_classification
from sklearn.preprocessing import  OneHotEncoder 
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, get_scorer_names

### 1.2 .csv files and description:

1.2.1 From data-description.txt:

* cap-shape:                bell=b,conical=c,convex=x,flat=f,
                            knobbed=k,sunken=s

* cap-color:                brown=n,buff=b,cinnamon=c,gray=g,green=r,
                            pink=p,purple=u,red=e,white=w,yellow=y

* bruises?:                 bruises=t,no=f

* stalk-color-above-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,
                            pink=p,red=e,white=w,yellow=y

* stalk-color-below-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,

* population:               abundant=a,clustered=c,numerous=n,
                            scattered=s,several=v,solitary=y

1.2.2 .csv:

In [2]:
train_csv = pd.read_csv("train.csv")
test_csv = pd.read_csv("test.csv")

1.2.3 bugfix

In [3]:
set_config(transform_output='pandas')
np.random.seed(1337)

## 2. Explorative Data Analysis:

In [4]:
# train_csv.info()

In [5]:
train_csv.set_index('Id', inplace=True)
#test_csv.set_index('Id', inplace=True) # fucked up later output so it´s in comment now

### 2.2 Sweetviz

In [6]:
train_report = sv.analyze(train_csv, target_feat= 'poisonous') 

                                             |          | [  0%]   00:00 -> (? left)

In [7]:
train_report.show_html(filepath='train_report.html', 
            open_browser=True, 
            layout='widescreen', 
            scale=None)

Report train_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### 2.3. EDA Conclusion

In [8]:
# Drop Cap Color(cap.color)and Bruises(bruises) since we want a secure prediction:
train_csv.drop(['cap.color','bruises'],axis=1, inplace=True)
test_csv.drop(['cap.color','bruises'],axis=1, inplace=True)

## 3. Doing the stuff...

In [9]:
X = train_csv.copy()
y = X.pop('poisonous')

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=1337) # higher train size to make up for not oversampling
# X_train = X_train.values
# y_train = y_train.values
rf = RandomForestClassifier()

In [10]:
pipe = make_pipeline((OneHotEncoder(sparse_output=False)),
                     (rf))
pipe

Pipeline(steps=[('onehotencoder', OneHotEncoder(sparse_output=False)),
                ('randomforestclassifier', RandomForestClassifier())])

In [11]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder', OneHotEncoder(sparse_output=False)),
                ('randomforestclassifier', RandomForestClassifier())])

In [12]:
param_grid = {'randomforestclassifier__n_estimators': [2, 3, 4],
              'randomforestclassifier__max_depth': [2, 3, 5]
              }


In [13]:
grid_search = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=5,
    verbose=1,
    scoring='roc_auc'
)

In [14]:
# sklearn.metrics.get_scorer_names()

In [15]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\_response.py", line 73, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklea

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('onehotencoder',
                                        OneHotEncoder(sparse_output=False)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             param_grid={'randomforestclassifier__max_depth': [2, 3, 5],
                         'randomforestclassifier__n_estimators': [2, 3, 4]},
             scoring='roc_auc', verbose=1)

In [16]:
grid_search.best_estimator_.fit(X,y)

Pipeline(steps=[('onehotencoder', OneHotEncoder(sparse_output=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=2, n_estimators=2))])

In [17]:

id_column = test_csv.pop('Id')
predictions = grid_search.best_estimator_.predict(test_csv)
results = pd.DataFrame({'Id':id_column,'poisonous':predictions})
results.to_csv('compition-file1.csv',index=False)

In [18]:
# results.shape

In [19]:
count = (results['poisonous'] == 0).sum()
print(count)

286
